In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.15)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 4.06 sec - Loss 0.060898 - ACC 60.75% - ACC Mean 60.75% - AUC 65.72% - AUC Mean 65.72% - Deter 000
Ite 00050 - 1.49 sec - Loss 0.012381 - ACC 60.60% - ACC Mean 60.51% - AUC 64.44% - AUC Mean 64.66% - Deter 011
Ite 00089 - 1.22 sec - Loss 0.007396 - ACC 60.70% - ACC Mean 60.33% - AUC 64.02% - AUC Mean 64.28% - Deter 050
Early stop ite 89, rollback to correction of ite 39, whith acc of 62.4% and auc of 67.28%
OUR METHOD RUN 1/10 - acc: 62.4% +- 0.0% - auc: 67.28% +- 0.0%
Ite 00000 - 1.64 sec - Loss 0.138130 - ACC 59.75% - ACC Mean 59.75% - AUC 65.04% - AUC Mean 65.04% - Deter 000
Ite 00050 - 1.14 sec - Loss 0.052407 - ACC 61.15% - ACC Mean 61.12% - AUC 65.75% - AUC Mean 65.31% - Deter 042
Ite 00058 - 0.98 sec - Loss 0.047915 - ACC 60.70% - ACC Mean 61.03% - AUC 65.36% - AUC Mean 65.26% - Deter 050
Early stop ite 58, rollback to correction of ite 8, whith acc of 62.75% and auc of 66.98%
OUR METHOD RUN 2/10 - acc: 62.75% +- 0.0% - auc: 66.98% +- 0.0%
Ite 00000 - 1.03 sec - Loss

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 61.2% +- 0.0% - auc: 64.88% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 60.65% +- 3.04% - auc: 64.84% +- 3.34%
SFIL RUN 2/10 - acc: 60.15% +- 1.02% - auc: 63.92% +- 2.24%
SFIL RUN 3/10 - acc: 60.0% +- 1.27% - auc: 63.51% +- 1.8%
SFIL RUN 4/10 - acc: 61.2% +- 2.65% - auc: 64.96% +- 3.17%
SFIL RUN 5/10 - acc: 59.75% +- 1.9% - auc: 62.91% +- 2.66%
SFIL RUN 6/10 - acc: 62.0% +- 2.57% - auc: 66.19% +- 2.14%
SFIL RUN 7/10 - acc: 61.25% +- 2.38% - auc: 65.89% +- 1.44%
SFIL RUN 8/10 - acc: 60.15% +- 1.89% - auc: 63.77% +- 1.68%
SFIL RUN 9/10 - acc: 59.5% +- 1.81% - auc: 63.24% +- 1.78%
SFIL RUN 10/10 - acc: 60.65% +- 2.47% - auc: 65.35% +- 2.55%
SFIL GLOBAL - acc: 60.53% +- 0.73% - auc: 64.46% +- 1.08%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 59.5% +- 1.47% - auc: 62.83% +- 1.18%
PFIL RUN 2/10 - acc: 59.5% +- 0.99% - auc: 64.88% +- 1.35%
PFIL RUN 3/10 - acc: 59.9% +- 1.94% - auc: 64.5% +- 2.36%
PFIL RUN 4/10 - acc: 59.85% +- 2.7% - auc: 64.44% +- 2.2%
PFIL RUN 5/10 - acc: 59.55% +- 1.53% - auc: 63.71% +- 2.2%
PFIL RUN 6/10 - acc: 60.7% +- 2.17% - auc: 64.78% +- 2.9%
PFIL RUN 7/10 - acc: 59.05% +- 1.62% - auc: 64.12% +- 2.37%
PFIL RUN 8/10 - acc: 60.85% +- 0.89% - auc: 64.55% +- 1.38%
PFIL RUN 9/10 - acc: 61.7% +- 2.55% - auc: 66.1% +- 2.56%
PFIL RUN 10/10 - acc: 59.8% +- 2.33% - auc: 64.3% +- 2.4%
PFIL GLOBAL - acc: 60.04% +- 0.76% - auc: 64.42% +- 0.8%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 58.9% +- 1.21% - auc: 62.62% +- 1.81%
SPOL RUN 2/10 - acc: 60.65% +- 2.03% - auc: 64.88% +- 2.55%
SPOL RUN 3/10 - acc: 60.85% +- 1.5% - auc: 63.86% +- 1.82%
SPOL RUN 4/10 - acc: 58.8% +- 2.47% - auc: 62.8% +- 2.7%
SPOL RUN 5/10 - acc: 60.25% +- 2.23% - auc: 64.3% +- 2.28%
SPOL RUN 6/10 - acc: 59.35% +- 1.83% - auc: 63.24% +- 1.31%
SPOL RUN 7/10 - acc: 59.05% +- 2.91% - auc: 63.79% +- 3.17%
SPOL RUN 8/10 - acc: 59.8% +- 1.77% - auc: 63.39% +- 2.94%
SPOL RUN 9/10 - acc: 59.3% +- 0.64% - auc: 63.54% +- 1.26%
SPOL RUN 10/10 - acc: 59.2% +- 1.34% - auc: 62.43% +- 2.05%
SPOL GLOBAL - acc: 59.61% +- 0.7% - auc: 63.48% +- 0.72%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 59.85% +- 2.15% - auc: 63.35% +- 2.63%
PPOL RUN 2/10 - acc: 60.0% +- 2.53% - auc: 64.45% +- 2.75%
PPOL RUN 3/10 - acc: 61.0% +- 2.09% - auc: 65.2% +- 1.97%
PPOL RUN 4/10 - acc: 60.35% +- 1.5% - auc: 63.42% +- 1.16%
PPOL RUN 5/10 - acc: 62.25% +- 2.25% - auc: 65.96% +- 2.65%
PPOL RUN 6/10 - acc: 59.8% +- 0.94% - auc: 64.27% +- 0.97%
PPOL RUN 7/10 - acc: 58.35% +- 2.22% - auc: 62.04% +- 2.11%
PPOL RUN 8/10 - acc: 60.6% +- 2.01% - auc: 65.88% +- 3.21%
PPOL RUN 9/10 - acc: 60.65% +- 0.6% - auc: 65.32% +- 1.61%
PPOL RUN 10/10 - acc: 61.2% +- 1.87% - auc: 64.08% +- 2.49%
PPOL GLOBAL - acc: 60.4% +- 0.98% - auc: 64.4% +- 1.18%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
